In [1]:
import os
import sys
import yaml
import logging
from models.ncsn_unet import (
  SinusoidalPosEmb,
  GaussianFourierProjection,
  Dense
)
sys.path.append('./nsf')
sys.path.append('./mintnet')
from nsf.nde import distributions, transforms, flows
import argparse

from models import utils
import torch
import torch.nn as nn
import pickle
from nsf.experiments.images_centering_copula import create_transform

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
ckpt_path = os.path.join('flow_ckpts', 'copula_best.pt')

# load data stats, no need to load a separate checkpoint
with open(os.path.join( 'flow_ckpts', 'data_means.p'), 'rb') as fp:
    data_stats = pickle.load(fp)
val_mean = data_stats['val_mean']

# stats
train_mean = data_stats['train_mean']
train_std = data_stats['train_std']

# doesn't matter, can just get it from the data_stats object
# from torchvision.datasets import MNIST
# from datasets import logit_transform
# import torchvision
# data_dir = '/atlas/u/kechoi/time-score-dre/'
# test_transform = torchvision.transforms.Compose([
#   torchvision.transforms.Resize(config.data.image_size),
#   torchvision.transforms.ToTensor()
# ])
# dataset = MNIST(os.path.join(data_dir, 'datasets', 'mnist_test'),
#                 train=False,
#                 download=True,
#                 transform=test_transform)
# data = dataset.data.unsqueeze(1).float()
# # dequantize
# data = (data + torch.rand_like(data)) / 256.
# data = logit_transform(data)
# val_mean = data.mean(0)  # lol will this make a diff? (no)

# annoying data transforms
c = 1
h = w = 28
spline_params = {
"apply_unconditional_transform": False,
"min_bin_height": 0.001,
"min_bin_width": 0.001,
"min_derivative": 0.001,
"num_bins": 128,
"tail_bound": 3.0
}
distribution = distributions.StandardNormal((c * h * w,)).cuda()
distribution = distribution

train_transform, val_transform, transform = create_transform(
  c, h, w, train_mean, val_mean, train_std, levels=2, hidden_channels=64,
  steps_per_level=8, alpha=0.000001, num_bits=8, preprocessing="realnvp_2alpha",
  multi_scale=False, actnorm=True, coupling_layer_type="rational_quadratic_spline",
  spline_params=spline_params, use_resnet=False, num_res_blocks=2,
  resnet_batchnorm=False, dropout_prob=0.0)
train_transform = train_transform.cuda()
val_transform = val_transform.cuda()
transform = transform.cuda()

# net = flows.Flow(transform, distribution)
net = flows.FlowDataTransform(transform, distribution, train_transform, val_transform).cuda()

# load checkpoint
checkpoint = torch.load(ckpt_path)
# net.load_state_dict(checkpoint)

# TODO: this is only for testing purposes! will go away if you train again
xs = {'_train_transform.'+k: v for k, v in train_transform.state_dict().items()}
ys = {'_val_transform.'+k: v for k, v in val_transform.state_dict().items()}
new_state_dict = {**checkpoint, **xs, **ys}
net.load_state_dict(new_state_dict)
net = net.to('cuda')
net=net.eval()


/net/galaxy/home/koes/rishal/mnist_priors/dre-infinity/nsf/nde/transforms/standard.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('_shift', torch.tensor(shift if (shift is not None) else 0.))
/tmp/ipykernel_2976339/2214980745.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are ex

In [4]:
sample,log_prob=net.sample_and_log_prob(1000)
print(sample.shape)
print((log_prob/sample.shape[1]).mean())

/net/galaxy/home/koes/rishal/.conda/envs/dre-infinity/lib/python3.10/site-packages/torch/functional.py:2135: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327489/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:1990.)
  return torch._lu_with_info(A, pivot=pivot, check_errors=(not get_infos))


torch.Size([1000, 784])
tensor(-1.3919)


RQ-NSF prior

In [30]:
ckpt_path = os.path.join( 'flow_ckpts', 'rq_nsf_best.pt')
print('loading model from checkpoint: {}'.format(ckpt_path))

# annoying data transforms
c = 1
h = w = 28
spline_params = {
"apply_unconditional_transform": False,
"min_bin_height": 0.001,
"min_bin_width": 0.001,
"min_derivative": 0.001,
"num_bins": 8,
"tail_bound": 3.0
}
distribution = distributions.StandardNormal((c * h * w,))
# TODO (HACK): get rid of hardcoding
from nsf.experiments.images import create_transform
transform = create_transform(c, h, w,
                                levels=2, hidden_channels=64, steps_per_level=8, alpha=0.000001,
                                num_bits=8, preprocessing="realnvp_2alpha", multi_scale=False,
                                actnorm=True, coupling_layer_type="rational_quadratic_spline",
                                spline_params=spline_params,
                                use_resnet=False, num_res_blocks=2, resnet_batchnorm=False, dropout_prob=0.0)

net = flows.Flow(transform, distribution)

# load checkpoint
checkpoint = torch.load(ckpt_path)
net.load_state_dict(checkpoint)
net = net.to('cuda')
net=net.eval()

loading model from checkpoint: flow_ckpts/rq_nsf_best.pt


/tmp/ipykernel_2976339/3839958577.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path)


In [35]:
sample,log_prob=net.sample_and_log_prob(1000)
print(sample.shape)
print((log_prob/(sample.shape[2]*sample.shape[3])).mean())

torch.Size([1000, 1, 28, 28])
tensor(-0.7671)


In [21]:
import pickle
from nsf.experiments.images_noise import create_transform

# load data stats, no need to load a separate checkpoint
with open(os.path.join( 'flow_ckpts', 'test_data_stats.p'), 'rb') as fp:
    data_stats = pickle.load(fp)

train_mean = data_stats['train_mean'].to('cuda')
test_mean = data_stats['test_mean'].to('cuda')
val_mean = data_stats['val_mean'].to('cuda')
train_cov = data_stats['train_cov_cholesky'].to('cuda')
val_cov = data_stats['val_cov_cholesky'].to('cuda')

# annoying data transforms
c = 1
h = w = 28
spline_params = {
"apply_unconditional_transform": False,
"min_bin_height": 0.001,
"min_bin_width": 0.001,
"min_derivative": 0.001,
"num_bins": 128,
"tail_bound": 3.0
}
distribution = distributions.StandardNormal((c * h * w,))
# TODO (HACK): get rid of hardcoding

train_transform, val_transform, transform = create_transform(
    c, h, w, train_mean, val_mean, train_cov, val_cov, levels=2, hidden_channels=64,
    steps_per_level=8, alpha=0.000001, num_bits=8, preprocessing="realnvp_2alpha",
    multi_scale=False, actnorm=True, coupling_layer_type="rational_quadratic_spline",
    spline_params=spline_params, use_resnet=False, num_res_blocks=2,
    resnet_batchnorm=False, dropout_prob=0.0)

# map this "flow" onto the device
# net = flows.Flow(transform, distribution)
net = flows.FlowDataTransform(transform, distribution, train_transform, val_transform)

net = net.to('cuda')

In [28]:
sample,log_prob=net.sample_and_log_prob(1000)
print(sample.shape)
print((log_prob/(sample.shape[2]*sample.shape[3])).mean())

torch.Size([1000, 1, 28, 28])
tensor(-1.7400)
